In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random
from wandb.keras import WandbCallback


import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision

## custom modules
import src.genformer1 as genformer
from src.layers.layers import *
import src.metrics as metrics
import src.optimizers as optimizers
import src.schedulers as schedulers
import src.utils as utils

import training_utils

2022-05-24 16:05:44.638595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 16:05:44.638655: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect('node-11') # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    
mixed_precision.set_global_policy('mixed_bfloat16')
with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism = 1
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 4
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2022-05-24 16:05:47.136830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-24 16:05:47.136890: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-24 16:05:47.136918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-05-24 16:05:47.427442: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-24 16:05:47.439223: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.44.184.242:8470}
2022-05-24 16:05:47.439282: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:37197}
2022-05-24 16:05:47.455539: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.44.184.242:8470}
2022-05-24 16:05:47.455591: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:37197}
2022-0

INFO:tensorflow:Initializing the TPU system: node-11


INFO:tensorflow:Initializing the TPU system: node-11


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
with strategy.scope():

    train_steps=400
    warmup_steps=10
    val_steps= 200 ### 5562
    num_epochs=40
    lr_base=0.0005
    warmup_lr=1.0e-04

    data_it_tr_list = []
    data_it_val_list = []

    ### create dataset iterators
    heads_dict = {}
    orgs = ["hg","mm"]
    for k, org in enumerate(orgs):
        heads_dict[org] = int(k)
    data_dict_tr, data_dict_val = training_utils.return_distributed_iterators(heads_dict,
                                                                              "gs://picard-testing-176520/TSS_164k_82kstride_ATAC_scaled_blacklist0.10_4096outputres_logTPM/preprocessed",
                                                                              GLOBAL_BATCH_SIZE,
                                                                              163840,
                                                                              20,
                                                                              8,
                                                                              num_epochs,
                                                                              strategy,
                                                                              options)


    

In [4]:
with strategy.scope():
    model = genformer.genformer(kernel_transformation="softmax_kernel_transformation",
                                dropout_rate=0.35,
                                final_out_length=20,
                                num_heads=4,
                                numerical_stabilizer=0.0000001,
                                nb_random_features=256,
                                hidden_size=96,
                                d_model=96,
                                dim=24,
                                max_seq_length = 4096,
                                rel_pos_bins=192,
                                widening = 2, ## ratio between first and second dense layer units in transformer block
                                conv_filter_size=15,
                                transformer_depth=3,
                                momentum=0.90,
                                channels_list=[24,24,24,24,48,48,48,48,96,96,96],
                                kernel_regularizer=0.001,
                                heads_dict=heads_dict)

In [5]:
with strategy.scope():

    optimizer = tfa.optimizers.AdaBelief(learning_rate=0.0007,
                                         weight_decay=1.0e-04,
                                         warmup_proportion=0.1,
                                         epsilon=1e-10,
                                         rectify=True,
                                         min_lr=1.0e-06,
                                         total_steps=4000)
    optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=6, slow_step_size=0.5)

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step, metric_dict = training_utils.return_train_val_functions_hg_mm(model,
                                                                                        optimizer,
                                                                                        strategy,
                                                                                        metric_dict, 
                                                                                        train_steps,
                                                                                        val_steps,
                                                                                        GLOBAL_BATCH_SIZE,
                                                                                        10.0) # last is gradient clip

In [7]:
with strategy.scope():
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, num_epochs):
        start = time.time()
        train_step(data_dict_tr['hg'],
                   data_dict_tr['mm'])
        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))
        val_step(data_dict_val['hg'],
                 data_dict_val['mm'])

        val_losses.append(metric_dict['hg_val'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_val_pearson: ' + str(metric_dict['hg_corr_stats'].result()['pearsonR'].numpy()))
        print('hg_val_R2: ' + str(metric_dict['hg_corr_stats'].result()['R2'].numpy()))
        
        print('mm_train_loss: ' + str(metric_dict['mm_tr'].result().numpy()))

        print('mm_val_loss: ' + str(metric_dict['mm_val'].result().numpy()))
        print('mm_val_pearson: ' + str(metric_dict['mm_corr_stats'].result()['pearsonR'].numpy()))
        print('mm_val_R2: ' + str(metric_dict['mm_corr_stats'].result()['R2'].numpy()))
        
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
    
        print('patience counter at: ' + str(patience_counter))

        for key, item in metric_dict.items():
            item.reset_state()

hg_train_loss: 0.2905462
hg_val_loss: 0.21245228
hg_val_pearson: 0.4399658
hg_val_R2: -0.54906523
mm_train_loss: 0.2603066
mm_val_loss: 0.20253713
mm_val_pearson: 0.52819574
mm_val_R2: -0.4305017
completed epoch 1
duration(mins): 3.4589707295099896
patience counter at: 0
hg_train_loss: 0.22135647
hg_val_loss: 0.17066729
hg_val_pearson: 0.43111372
hg_val_R2: -1.223413
mm_train_loss: 0.18181624
mm_val_loss: 0.17141688
mm_val_pearson: 0.5020159
mm_val_R2: -0.8994833
completed epoch 2
duration(mins): 1.2804106871287029
patience counter at: 0
hg_train_loss: 0.22719371


KeyboardInterrupt: 

In [8]:
model.save_weights("gs://picard-testing-176520/genformer_test_model22/checkpoint")